<a href="https://colab.research.google.com/github/RubenGres/food-dysmorphia/blob/main/LangChain_link_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community tavily-python langchain-openai

In [5]:
import os
import json
import time
from openai import OpenAI
from tavily import TavilyClient

os.environ["OPENAI_API_KEY"] = "sk-123456"
os.environ["TAVILY_API_KEY"]= "tvly-123456"

# Initialize clients with API keys
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

assistant_prompt_instruction = """
Your goal is to provide answers based on information from the internet.
You must use the provided Tavily search API function to find relevant online information.
You should never use your own knowledge to answer questions.
Please include relevant url sources in the end of your answers.
Answer the question based only on the context provided.
Return as many paragraphs as there are urls in the context.

For each url in the context, the response format is:
<anwser n>
Source: <url n>
"""

# Function to perform a Tavily search
def tavily_search(query):
    search_result = tavily_client.get_search_context(query, search_depth="advanced", max_tokens=8000)
    return search_result

# Function to wait for a run to complete
def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

# Function to handle tool output submission
def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "tavily_search":
            output = tavily_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

# Function to print messages from a thread
def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

# Create an assistant
assistant = client.beta.assistants.create(
    instructions=assistant_prompt_instruction,
    model="gpt-4-1106-preview",
    tools=[{
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Get information on recent events from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query to use. For example: 'Latest news on Nvidia stock performance'"},
                },
                "required": ["query"]
            }
        }
    }]
)
assistant_id = assistant.id
print(f"Assistant ID: {assistant_id}")

# Create a thread
thread = client.beta.threads.create()
print(f"Thread: {thread}")

# Ongoing conversation loop
user_input = "How to improve profits on lemons ?"

# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_input,
)

# Create a run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

# Wait for run to complete
run = wait_for_run_completion(thread.id, run.id)

if run.status == 'failed':
  print(run.error)

elif run.status == 'requires_action':
  run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
  run = wait_for_run_completion(thread.id, run.id)

# Print messages from the thread
print_messages_from_thread(thread.id)

Assistant ID: asst_hhgj09JOmoB42NwwzuddlX2W
Thread: Thread(id='thread_XsqObru9dSIDULHmOsFFIzOx', created_at=1712770874, metadata={}, object='thread')
assistant: Improving profits on lemons involves a combination of efficient farming techniques and strategic sales approaches. Here are some key points to consider:

1. Implement efficient farming techniques, such as proper land preparation, effective irrigation systems, pest control strategies, and regular pruning to increase yields and maintain healthy lemon trees. Embracing sustainable and organic practices can also help you command higher prices in the market.
Source: https://finmodelslab.com/blogs/profitability/lemon-cultivation-profitability

2. Fertilize lemon trees appropriately using a balanced fertilizer, like a 10-10-10 mix, during the growing season to provide essential nutrients. Organic fertilizers such as compost, manure, or fish emulsion can also be beneficial.
Source: https://www.roysfarm.com/lemon-farming/

3. Prolong the